In [48]:
# 필요한 utils
import pandas as pd
import numpy as np

import sys
sys.path.append("../3.src")
from utils import *

In [49]:
import importlib
import utils
importlib.reload(utils)


<module 'utils' from '/data/ephemeral/home/workspace/upstageailab-ml-competition-ml-2/2.Notebook/../3.src/utils.py'>

In [40]:
# 3. csv 파일 불러오기
DATA_PATH = "/data/ephemeral/home/workspace/upstageailab-ml-competition-ml-2/1.Data/"
train_df = pd.read_csv(DATA_PATH + "train.csv")
test_df = pd.read_csv(DATA_PATH + "test.csv")

subway_df = pd.read_csv(DATA_PATH + "subway_feature.csv")
bus_df = pd.read_csv(DATA_PATH + "bus_feature.csv")

/tmp/ipykernel_630080/517515685.py:3: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(DATA_PATH + "train.csv")


In [41]:
# train 데이터와 test 데이터 하나로 합치기! 
train_df['is_train'] = 1
test_df['is_train'] = 0
all_df = pd.concat([train_df, test_df], ignore_index= True)

In [42]:
all_df['full_address'] = all_df['시군구'].astype(str) + " " + all_df['도로명'].astype(str)

In [72]:
# 1. 결측치인 행 중 유니크한 주소만
missing_mask = all_df['좌표X'].isnull() | all_df['좌표Y'].isnull()
unique_missing_addresses = all_df.loc[missing_mask, ['full_address', '좌표X', '좌표Y','도로명']].drop_duplicates().reset_index(drop=True)

In [62]:
def get_lat_lon_from_naver_3(address, api_key_id, api_key):
    url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": api_key_id,
        "X-NCP-APIGW-API-KEY": api_key,
        "Accept": "application/json"
    }
    params = {
        "query": address
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: HTTP {response.status_code}")
        return None, None

    data = response.json()

    # 정상 응답인지 체크
    if data.get("status") != "OK":
        print("API Error:", data.get("errorMessage"))
        return None, None

    addresses = data.get("addresses")
    if not addresses:
        print("No geocode result found.")
        return None, None

    # 첫 번째 결과에서 x,y 추출
    x = addresses[0].get("x")
    y = addresses[0].get("y")
    return y, x

In [ ]:
print(get_lat_lon_from_naver_3("언주로 110","***", "***"))

('37.4848777', '127.0542279')


In [69]:
def get_lat_lon_from_naver_df_2(original_df, api_key_id, api_key):
    new_df = original_df.copy()
    tqdm.pandas()

    print("start")
    # --------- 주소 → 위도/경도 호출 함수 -----------
    def geocode_address(address):
        url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": api_key_id,
            "X-NCP-APIGW-API-KEY": api_key,
            "Accept": "application/json"
        }
        params = {"query": address}

        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code != 200:
                print(f"Error: HTTP {response.status_code} for address {address}")
                return pd.Series({'좌표X': None, '좌표Y': None})

            data = response.json()

            if data.get("status") != "OK":
                print("API Error:", data.get("errorMessage"))
                return pd.Series({'좌표X': None, '좌표Y': None})

            addresses = data.get("addresses")
            if not addresses:
                print("No geocode result found for:", address)
                return pd.Series({'좌표X': None, '좌표Y': None})

            x = addresses[0].get("x")
            y = addresses[0].get("y")
            return pd.Series({'좌표X': x, '좌표Y': y})

        except Exception as e:
            print(f"Error geocoding {address}: {e}")
            return pd.Series({'좌표X': None, '좌표Y': None})

    # --------- 좌표 결측치 있는 행만 처리 -----------
    mask = new_df['좌표X'].isnull() | new_df['좌표Y'].isnull()

    if mask.sum() > 0:
        coords = new_df.loc[mask, '도로명'].progress_apply(geocode_address)
        coords.index = new_df.loc[mask].index
        new_df.loc[mask, ['좌표X', '좌표Y']] = coords
    print("end")
    return new_df

In [73]:
unique_missing_addresses

,full_address,좌표X,좌표Y,도로명
0,서울특별시 강남구 개포동 삼성로 14,NaN,NaN,삼성로 14
1,서울특별시 강남구 개포동 개포로 516,NaN,NaN,개포로 516
2,서울특별시 강남구 개포동 언주로 105,NaN,NaN,언주로 105
3,서울특별시 강남구 개포동 언주로 110,NaN,NaN,언주로 110
4,서울특별시 강남구 개포동 논현로2길 61-4,NaN,NaN,논현로2길 61-4
...,...,...,...,...
8508,서울특별시 노원구 월계동 덕릉로60길 268,NaN,NaN,덕릉로60길 268
8509,서울특별시 동작구 흑석동 서달로 90,NaN,NaN,서달로 90
8510,서울특별시 마포구 신수동 토정로17길 6,NaN,NaN,토정로17길 6
8511,서울특별시 성동구 송정동 송정길 32,NaN,NaN,송정길 32


In [ ]:
new_df = get_lat_lon_from_naver_df_2(unique_missing_addresses,"***", "***")

start


  0%|          | 11/8513 [00:03<45:46,  3.10it/s]

No geocode result found for: 언주로 21


  4%|▍         | 328/8513 [01:49<43:35,  3.13it/s]

No geocode result found for: 언주로70길 24


...

No geocode result found for: 송파대로48길 


100%|█████████▉| 8501/8513 [47:47<00:03,  3.24it/s]

API Error: query is INVALID


100%|██████████| 8513/8513 [47:52<00:00,  2.96it/s]

end


In [77]:
new_df.head()

,full_address,좌표X,좌표Y,도로명
0,서울특별시 강남구 개포동 삼성로 14,127.0716058,37.4851086,삼성로 14
1,서울특별시 강남구 개포동 개포로 516,127.0704418,37.4886321,개포로 516
2,서울특별시 강남구 개포동 언주로 105,127.0528961,37.4816632,언주로 105
3,서울특별시 강남구 개포동 언주로 110,127.0542279,37.4848777,언주로 110
4,서울특별시 강남구 개포동 논현로2길 61-4,127.0548197,37.4748238,논현로2길 61-4


In [78]:
merged = all_df.merge(
    new_df[['도로명', '좌표X', '좌표Y']], 
    on='도로명', 
    how='left', 
    suffixes=('', '_new')
)

merged['좌표X'] = merged['좌표X'].fillna(merged['좌표X_new'])
merged['좌표Y'] = merged['좌표Y'].fillna(merged['좌표Y_new'])



In [83]:
all_df[all_df['좌표X'].isnull()].index

Int64Index([    975,     976,     977,     978,     979,     980,     981,
                982,     983,     984,
            ...
            1128070, 1128071, 1128072, 1128073, 1128074, 1128075, 1128076,
            1128077, 1128078, 1128079],
           dtype='int64', length=876232)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_train,full_address,좌표X_new,좌표Y_new
975,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,127.0716058,37.4851086,NaN,134000.0,1,서울특별시 강남구 개포동 삼성로 14,127.0716058,37.4851086
976,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,4,1982,...,NaN,NaN,127.0716058,37.4851086,NaN,158000.0,1,서울특별시 강남구 개포동 삼성로 14,127.0716058,37.4851086
977,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,127.0716058,37.4851086,NaN,155000.0,1,서울특별시 강남구 개포동 삼성로 14,127.0716058,37.4851086
978,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,42.55,201801,26,4,1982,...,NaN,NaN,127.0716058,37.4851086,NaN,132000.0,1,서울특별시 강남구 개포동 삼성로 14,127.0716058,37.4851086
979,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201802,10,1,1982,...,NaN,NaN,127.0716058,37.4851086,NaN,154000.0,1,서울특별시 강남구 개포동 삼성로 14,127.0716058,37.4851086


In [84]:
final_result_df = merged.drop(columns=['좌표X_new', '좌표Y_new'])

In [ ]:
final_result_df['']['좌표X'].isnull().sum()

30731

In [87]:
final_result_df.to_csv('/data/ephemeral/home/workspace/upstageailab-ml-competition-ml-2/1.Data/naver_api_XY.csv', index=False)

In [ ]:


# 2. 지오코딩
geocoded_df = get_lat_lon(unique_missing_addresses)



In [ ]:
# 3. 병합 후 좌표 채우기
all_df = all_df.merge(
    geocoded_df,
    on='full_address',
    how='left',
    suffixes=('', '_new')
)
all_df['좌표X'] = all_df['좌표X'].fillna(all_df['좌표X_new'])
all_df['좌표Y'] = all_df['좌표Y'].fillna(all_df['좌표Y_new'])
all_df.drop(columns=['좌표X_new', '좌표Y_new'], inplace=True)

In [11]:
# all_df['도로명'].isnull().sum()
# all_df['시군구'].isnull().sum()

0

In [82]:
# 좌표 라이브러리 추가해서 결측치 보강!!!!
print(all_df['좌표X'].isnull().sum(), all_df['좌표Y'].isnull().sum())


876232 876232


In [83]:
null_idx = all_df[all_df['좌표X'].isnull()].head(5).index
print(null_idx)
all_df.loc[null_idx, '좌표X']

Int64Index([975, 976, 977, 978, 979], dtype='int64')


975   NaN
976   NaN
977   NaN
978   NaN
979   NaN
Name: 좌표X, dtype: float64

In [84]:
lat_test_df= all_df[all_df['좌표X'].isnull()].head(5)
lat_test_df

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_train
975,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134000.0,1
976,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,4,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158000.0,1
977,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155000.0,1
978,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,42.55,201801,26,4,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132000.0,1
979,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201802,10,1,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154000.0,1


In [85]:
import sys
sys.stdout.flush()

In [86]:

new_df = get_lat_lon(lat_test_df)

100%|██████████| 5/5 [00:06<00:00,  1.26s/it]


In [87]:
print(type(new_df))
new_df.head()


<class 'pandas.core.frame.DataFrame'>


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_train,full_address
975,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,NaN,NaN,127.06956,37.483705,NaN,134000.0,1,서울특별시 강남구 개포동 삼성로 14
976,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,4,1982,...,NaN,NaN,NaN,NaN,127.06956,37.483705,NaN,158000.0,1,서울특별시 강남구 개포동 삼성로 14
977,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,NaN,NaN,127.06956,37.483705,NaN,155000.0,1,서울특별시 강남구 개포동 삼성로 14
978,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,42.55,201801,26,4,1982,...,NaN,NaN,NaN,NaN,127.06956,37.483705,NaN,132000.0,1,서울특별시 강남구 개포동 삼성로 14
979,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201802,10,1,1982,...,NaN,NaN,NaN,NaN,127.06956,37.483705,NaN,154000.0,1,서울특별시 강남구 개포동 삼성로 14


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_train,full_address
975,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134000.0,1,서울특별시 강남구 개포동 삼성로 14
976,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,4,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158000.0,1,서울특별시 강남구 개포동 삼성로 14
977,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201801,25,1,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155000.0,1,서울특별시 강남구 개포동 삼성로 14
978,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,42.55,201801,26,4,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132000.0,1,서울특별시 강남구 개포동 삼성로 14
979,서울특별시 강남구 개포동,189,189.0,0.0,개포주공4단지,50.39,201802,10,1,1982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154000.0,1,서울특별시 강남구 개포동 삼성로 14


In [16]:
new_all_df = get_lat_lon(all_df)

  0%|          | 347/876232 [09:04<544:24:51,  2.24s/it]

Error geocoding 서울특별시 강남구 논현동 학동로43길 38: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EA%B0%95%EB%82%A8%EA%B5%AC+%EB%85%BC%ED%98%84%EB%8F%99+%ED%95%99%EB%8F%99%EB%A1%9C43%EA%B8%B8+38&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


  0%|          | 350/876232 [09:11<628:20:47,  2.58s/it]

Error geocoding 서울특별시 강남구 논현동 학동로 338: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EA%B0%95%EB%82%A8%EA%B5%AC+%EB%85%BC%ED%98%84%EB%8F%99+%ED%95%99%EB%8F%99%EB%A1%9C+338&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


  0%|          | 367/876232 [09:41<599:01:48,  2.46s/it]

...

Error geocoding 서울특별시 강남구 논현동 학동로7길 6: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EA%B0%95%EB%82%A8%EA%B5%AC+%EB%85%BC%ED%98%84%EB%8F%99+%ED%95%99%EB%8F%99%EB%A1%9C7%EA%B8%B8+6&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


  0%|          | 406/876232 [11:07<914:53:06,  3.76s/it]

Error geocoding 서울특별시 강남구 논현동 강남대로146길 28: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EA%B0%95%EB%82%A8%EA%B5%AC+%EB%85%BC%ED%98%84%EB%8F%99+%EA%B0%95%EB%82%A8%EB%8C%80%EB%A1%9C146%EA%B8%B8+28&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


  0%|          | 533/876232 [14:29<396:47:54,  1.63s/it]


KeyboardInterrupt: 